In [65]:
from typing import List,TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langgraph.graph import StateGraph, END, START, add_messages
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
# create the subgraph state
class child_state(TypedDict):
    messages : Annotated[List, add_messages]

In [67]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm_with_tools = llm.bind_tools(tools = tools)

# llm_with_tools.invoke("what is the weather of bangalore?")

In [68]:
def child_search_agent(state: child_state) -> child_state:
    """
    This function is the child state that performs a search using the TavilySearchResults tool.
    It takes the current state, performs a search, and returns the updated state with the search results.
    """
    print("Child search agent invoked")
    # if 'conversation' not in state:
    #     state['conversation'] = []
    state['messages'].append(llm_with_tools.invoke(state['messages']))
    print(state)
    return state

In [69]:
def tools_router(state: child_state):
    """
    This function routes the conversation to the appropriate tool based on the content of the last message.
    If the last message contains 'search', it routes to the search tool; otherwise, it returns END.
    """
    print("Tools router invoked")
    last_message =  state['messages'][-1]
    print(f"Last message: {last_message}")
    if hasattr(last_message, 'tool_calls') and len(last_message.tool_calls) > 0:
        print("Routing to tool node")
        return 'tool_node'
    else:
        return END

In [70]:
tool_node = ToolNode(tools=tools)

In [71]:
subgraph = StateGraph(child_state)
subgraph.add_node("child_search_agent", child_search_agent)
subgraph.add_node('tool_node',tool_node)

subgraph.set_entry_point("child_search_agent")

subgraph.add_conditional_edges("child_search_agent",tools_router)
subgraph.add_edge("tool_node", "child_search_agent")

search_app = subgraph.compile()

# search_app.invoke({'messages': [HumanMessage("what is the weather of bangalore?")]})

In [72]:
class ParentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

parent_graph = StateGraph(ParentState)
parent_graph.add_node("search_app", search_app)

parent_graph.add_edge(START, "search_app")
parent_graph.add_edge("search_app", END)

parent_app = parent_graph.compile()
# parent_app.invoke({'messages': [HumanMessage("what is the weather of bangalore?")]})

In [73]:
# parent graph with different schema

class ParentStatewithdifferentSchema(TypedDict):
    query: str
    response: str

def search_agent(state: ParentStatewithdifferentSchema) -> ParentStatewithdifferentSchema:
    """
    This function is the search agent that performs a search based on the query in the state.
    It updates the response in the state with the result of the search.
    """
    subgraph_input = {
        'messages': [HumanMessage(content = state['query'])]
    }

    subgraph_output = search_app.invoke(subgraph_input)

    state['response'] = subgraph_output['messages'][-1].content

    return state


parent_graph_with_different_schema = StateGraph(ParentStatewithdifferentSchema)
parent_graph_with_different_schema.add_node("search_agent", search_agent)
parent_graph_with_different_schema.add_edge(START, "search_agent")
parent_graph_with_different_schema.add_edge("search_agent", END)

parent_app = parent_graph_with_different_schema.compile()

parent_app.invoke({'query': "what is the weather of bangalore?",'response': ""})

Child search agent invoked
{'messages': [HumanMessage(content='what is the weather of bangalore?', additional_kwargs={}, response_metadata={}, id='32ec9c6e-5c89-4bd6-99a4-9b8192a1a359'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZC9KlDdOD9tje6CvOJuE6lYY', 'function': {'arguments': '{"query":"Bangalore weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BglMX6U5Wvbr10mmWO3Qorhn94Rq8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5789d004-2146-4ee1-a55e-d3a82a1a02b3-0', tool_calls=[{'na

{'query': 'what is the weather of bangalore?',
 'response': "Today's weather in Bangalore is expected to be partly cloudy. The maximum temperature is likely to reach 31°C, while the minimum temperature might hover around 21°C. The wind speed is around 5.66 km/h, with gusts up to 10.24 km/h."}